#### TARTU ÜLIKOOL : Arvutiteaduse instituut : Informaatika õppekava
#### Autor: Raul Tölp
### Töö nimetus: Programmeerimise kursuse tagasiside analüüs
#### Bakalaureusetöö analüüsi osa
##### Juhendaja: Reimo Palm
#### Tartu 2020

### Materjalid: 2018 ja 2019 aasta moodle küsitlused
* Stardiküsitlus
* Lõpuküsitlus
* Kodutööd 12 tk
* Kontrolltööd 2 tk
* Hinneteleht
### lisaks:
* Moodles enesekontrolli testid

# Impordid

In [ ]:
import pandas as pd
import numpy as np
import re
import random

import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.ticker as mtick

# pd.options.display.max_colwidth = 100

# help(pd.Series.loc)

# Kasutatud lingid:
# https://www.listendata.com/2019/06/matplotlib-tutorial-learn-plot-python.html
# http://enhancedatascience.com/2017/04/15/linear-regression-python-predicting-car-sales-price/
# https://stackabuse.com/linear-regression-in-python-with-scikit-learn/
# https://www.datacamp.com/community/tutorials/wordcloud-python
# https://stackoverflow.com/questions/42818361/how-to-make-two-plots-side-by-side-using-python
# https://stackoverflow.com/questions/14770735/how-do-i-change-the-figure-size-with-subplots
# https://towardsdatascience.com/10-python-pandas-tricks-to-make-data-analysis-more-enjoyable-cb8f55af8c30
# https://matplotlib.org/gallery/api/two_scales.html
# https://matplotlib.org/3.1.1/gallery/subplots_axes_and_figures/secondary_axis.html
# https://stackoverflow.com/questions/43149703/adding-a-second-y-axis-related-to-the-first-y-axis
# https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.set_xticklabels.html
# https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.xticks.html
# https://stackoverflow.com/questions/13052844/matplotlib-how-to-decrease-density-of-tick-labels-in-subplots
# https://stackoverflow.com/questions/14770735/how-do-i-change-the-figure-size-with-subplots
# https://www.tutorialgateway.org/pandas-dataframe-plot/
# https://www.shanelynn.ie/bar-plots-in-python-using-pandas-dataframes/
# https://www.listendata.com/2019/06/matplotlib-tutorial-learn-plot-python.html
# https://python-graph-gallery.com/82-marginal-plot-with-seaborn/
# https://pythonspot.com/matplotlib-scatterplot/
# https://realpython.com/pandas-groupby/

In [ ]:
# https://stackoverflow.com/questions/18770504/resize-ipython-notebook-output-window
# %%html
# <style>
# .output_wrapper, .output {
#     height:auto !important;
#     max-height:400px;  /* your desired max-height here */
# }
# .output_scroll {
#     box-shadow:none !important;
#     webkit-box-shadow:none !important;
# }
# </style>

# OSA 1

# Üldandmed

In [ ]:
aastad = ['2018', '2019'] # kastutavad aastad
lehed = ['Start', '2', '3', '4', '5', 'KT1', '7', '8', '9', '10', '11', 'KT2', '13', '14', '15', 'Lõpp', 'Hinded'] # alamlehtede nimetused
vastanud = [] # abimuutuja df tegemiseks - vastanute arv
mõõtmed = [] # abimuutuja df tegemiseks - tabeli mõõtmed
andmed = [] # abimuutuja df tegemiseks - andmed

# Üldine info tabelite kohta
for aasta in aastad:
    xlsx = pd.ExcelFile(aasta + '.xlsx')
    vast = {}
    mõõt = {}
    andm = {}
    
    for leht in lehed:
        df = pd.read_excel(xlsx, leht) # loeme sisse alamlehe
        vast[leht] = df.shape[0] # vastanute arv
        mõõt[leht] = df.shape # lehe mõõtmed
        df_andmed = df.agg(['nunique','count','size']).T # iga tulba kohta unikaalsed, andmete hulk, täispikkus
        df_andmed['vastanute %'] = round(df_andmed['count'] / df_andmed['size'] * 100 , 1) # lisame vatanute protsendi tulba
        andm[leht] = df_andmed # lisame andmed sõnaraamatusse
        
        if leht == lehed[-1]: # Hinded - kui on hinnete leht, lisame andmete koguse nende kohta, kelle punktid > 0
            vast['Punktid > 0'] = df[df['Hinne (Punktid)'] != 0].shape[0]
        
    vastanud.append(vast)
    mõõtmed.append(mõõt)
    andmed.append(andm)

df_vastanud = pd.DataFrame(vastanud, index = aastad) # df vastanute arvust
df_mõõtmed = pd.DataFrame(mõõtmed, index = aastad) # df tabeli mõõtmetest
df_andmed = pd.DataFrame(andmed, index = aastad) # df kõikidest andmetest

In [ ]:
# df_mõõtmed - tabelite mõõtmed

print("Tabeleid ühes aastas:", len(lehed))
display(df_mõõtmed)

In [ ]:
# df_andmed - suur hulk tabeleid baasandmete kohta
# siin näha küsimustiku kusimused, nende arv (size), unikaalsete vastuste ja kõigi vastanute arv (count) tulpade kaupa
# võib panna lisasse, kui see mõistlik (samas pole väga oluline, lihtalt see abiks analüüsi tegemisel)

for index, aasta in enumerate(aastad):
    for nädal in lehed:
        print('Aasta:', aasta + ', küsitlus (nädal):', nädal)
        display(df_andmed.iloc[index][nädal])

In [ ]:
# df_vastanud - vastanute arv tabelites
# Selle jätsin vaid seetõttu, et hea võrdlus 'Lõpppunktid > 0' suhtes

print('Andmeridade arv erinevates tabelites:')
display(df_vastanud)

df_temp = df_vastanud.rename(columns = {'Punktid > 0': 'P>0'})
df_temp.drop(['Hinded'], axis=1, inplace=True)

# V1
# plt.figure(figsize=(8,4))
# plt.plot(df_temp.T)
# plt.legend(aastad, loc = "best")
# plt.ylabel('vastanute arv')
# plt.xlabel('Küsimustik (nädal)')
# plt.title('Andmeridade arv')
# plt.show()

# V2 - kuna stardiküsitluste andmed enamvähem sama suured, siis lisatud ka protsenditelg
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(df_temp.T, marker = '.')
ax2 = ax.twinx() # teeme teise y telje
Y2 = lambda z: z / df_temp.max().max() * 100 # max suhtes; saab ka [y-2], kus y = list(ax.get_yticks())
ymin, ymax = ax.get_ylim()
ax2.set_ylim((Y2(ymin), Y2(ymax)))
ax2.yaxis.set_major_formatter(mtick.PercentFormatter(decimals = 0))
ax2.locator_params(nbins = 20, axis = 'y')
ax.legend(aastad, loc = 'best')
ax.set_ylabel('vastanute arv')
ax.set_xlabel('Küsimustik (nädal)')
plt.title('Andmeridade arv')
plt.show()

In [ ]:
# Vastanute arv protsentides stardiküsitluse koguarvu suhtes
protsendid = []

for index, aasta in enumerate(aastad):
    andmed = {lehed[0]: 100} # Start saab 100%
    for i in range(1, len(lehed) - 1): # Starti ja Hindeid ei vaata
        andmed[lehed[i]] = round(vastanud[index][lehed[i]] / vastanud[index][lehed[0]] * 100, 1)
    protsendid.append(andmed)
    
print('Vastanute arv protsentides stardiküsitluse koguarvu suhtes:')
df_protsendid = pd.DataFrame(protsendid, index = aastad)
display(df_protsendid)

def plot_protsendid(df):
    fig,ax = plt.subplots(figsize=(10,5))
    x = df.columns

    for i in range(len(aastad)):
        data = ax.plot(x, df.iloc[i], label = df.index[i], marker = '.')
#         y = [df.iloc[i].mean()] * len(x)
#         mean = ax.plot(x, y, linestyle = '--', label = df.index[i] + ' keskmine') # keskmise joon
#         y = [df.iloc[i].median()] * len(x)
#         mean = ax.plot(x, y, linestyle = '-.', label = df.index[i] + ' mediaan') # mediaan, see ei anna suurt midagi
    
    ax.locator_params(nbins = 20, axis = 'y')
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals = 0))
    ax.legend(loc = 'best')
    plt.ylabel('vastanute arv protsentides')
    plt.xlabel('Küsimustik (nädal)')
    plt.title('Küsimustikele tagasiside andnute arv protsentides (start = 100%)')
    plt.show()
    
plot_protsendid(df_protsendid)

In [ ]:
# Kõikie küsitluste vastanute arv protsentides

for index, aasta in enumerate(aastad):
    fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(12,8), constrained_layout=True)

    for i in range(0, 4):
        for j in range(0, 4):
            nädal = i * 4 + j
            df_temp = pd.DataFrame(df_andmed.iloc[index][nädal][3:]['vastanute %']) # 4.-ndast tulbast hakkavad küsimused/vastused
            df_temp.index = np.arange(1, len(df_temp) + 1) # Küsimuste nimed numbriteks
            ax[i,j].plot(df_temp)
            ax[i,j].set_ylim(-5, 105)
            ax[i,j].locator_params(nbins = 10, axis = 'y')
            ax[i,j].yaxis.set_major_formatter(mtick.PercentFormatter(decimals = 0))
#                 ax[i,j].xaxis.set_ticks(df_temp.index)
            ax[i,j].title.set_text('Nädal: ' + lehed[nädal])

    fig.suptitle(aasta + ' nädalaküsitlustele vastanute arv protsentuaalselt küsimuste kaupa')
    plt.show()

# OSA 2

# Stardiküsitlus 2018-2019

In [ ]:
# COL	3	Kuidas hindad oma programmeerimisoskusi / teadmisi enne selle kursuse algust?
# COL	4	Täpsustus: kui oled programmeerimisega kokku puutunud, siis kuidas?
# COL	5	Oleta, millistes rollides võiksid sa tulevikus oma programmeerimisoskust otseselt või kaudselt rakendada. Vali need, mis tunduvad sulle enam-vähem realistlikud.
# COL	6	Mida see programm võiks teha ja miks sa nii arvad? Tegemist on väljamõeldud programmeerimiskeelega, seetõttu pole siin ei õiget ega valet vastust.
# COL	7	Mida sa kursuselt ootad? Kas arvad, et saab olema kerge? Raske? Igav? Huvitav? Miks? Midagi muud, mida soovid meile öelda? Siin saad ka täpsustada või kommenteerida eelmiste küsimuste vastuseid.

Tulemusi aanlüüsitakse anonüümselt ja grupeerimiseks kasutatakse stardiküsitluses antud eelenvat kogemust

In [ ]:
# Loeme sisse start ja hinnete lehed ja liidame kokku üheks DF-ks
# ei loeta neid, kellel puudub rühm ja seega ka hilisemad andmed
# sai kontrollitud, et ei oleks mitut sama nimega
# Lisainfo DF kohta: df.info(), df.describe(), df.count()

def loe_andmed(leht, lisa_hinded = False):
    df = {}
    for aasta in aastad:
        xlsx = pd.ExcelFile(aasta + '.xlsx')
        df_l = pd.read_excel(xlsx, leht)
        df_l = df_l.fillna('Andmed puuduvad')
        if lisa_hinded:
            df_h = pd.read_excel(xlsx, 'Hinded')
            df_h = df_h.replace(to_replace = '-', value = 0)
            df_h['Kasutaja täisnimi'] = df_h.Eesnimi + ' ' + df_h.Perenimi
            df[aasta] = pd.merge(df_l, df_h, on = 'Kasutaja täisnimi', how='left').fillna(0) # ja paneme need kokku
        else:
            df[aasta] = df_l
#         len(df[aasta][df[aasta].duplicated(['Kasutaja täisnimi'])]) # duplikaatide kontroll
    return df

def plot_barh_perc(df, tiitel, xtiitel = '', suurus = (8, 4), droppida = True, järjestus = '', skaala = 0):
    if len(list(df.index)) == len(järjestus):
        df = df.reindex(järjestus)
    elif len(järjestus) > 0:
        print('Sortimise viga:\n', list(df.index), '\n', järjestus)
        
    if droppida:
        dff = df.drop(aastad, axis=1, inplace=False)
    else:
        dff = df
        
    ax = dff.plot(kind = 'barh', figsize = suurus)
    ax.xaxis.set_major_formatter(mtick.PercentFormatter(decimals = skaala))
    ax.invert_yaxis() # et andmed oleks ülevalt alla kahanevas järjestuses ja väiksem aastaarv eespool
#     ax.locator_params(tight = True, axis = 'x') # rohkem ticke, kuid vahepeal teeb trikke
    plt.legend(loc='best', fancybox=True, framealpha=0.5)
    plt.xlabel(xtiitel)
    plt.title(tiitel)
    plt.show()
    display(df)

df_start = loe_andmed('Start', True) # stardiküsitlust + hinded ühes df-s
df_startstat = pd.DataFrame()

for aasta in aastad:
    df = df_start[aasta]
    start = len(df)
    r = len(df[df['Rühmad'] != 0])
    p = len(df[df['Hinne (Punktid)'] != 0])
    p50 = len(df[df['Hinne (Punktid)'] >= 50])
    df_startstat[aasta] = [start, r, p, p50]
    df_startstat[aasta + ' %'] = [100, round(r / start * 100, 1), round(p / start * 100, 1), round(p50 / start * 100, 1)]
    df_startstat.index = ['Stardiküsitlusele vastanuid', 'Rühm määratud', 'Punktid > 0', 'Punktid $\geq$ 50']

plot_barh_perc(df_startstat.drop(['Rühm määratud']), 'Võrdlused stardiküsitlusega')

In [ ]:
# üldine tulba statistika %-des
def col_stat(dfs, tulp, tükelda = False, sordi_index = False):
    data = {}
    tulba_nimi = dfs[aastad[0]].columns[tulp]
    for aasta in aastad:
        df = dfs[aasta]
        df = df.replace(['0', 0], np.nan) # 0 andmed asendame np.nan
        df = df.fillna("Andmed puuduvad") # np.nan asendame, et andmed puuduvad
        if tükelda: # kui ühes lahtris on mitmed andmed, siis tükeldame
            tükid = []
            for item in df[tulba_nimi]:
                tükid += item.split('\n')
                data[aasta] = pd.Series(Counter(tükid))
        else:
            data[aasta] = df[tulba_nimi].value_counts() # saab ka df.groupby(tulp)[tulp].agg('count')
            
        data[aasta + ' %'] = round(data[aasta] / data[aasta].sum() * 100, 1)
#         data[aasta] = map(int, list(data[aasta]))

    df = pd.DataFrame(data)
    
    if sordi_index:
        df.sort_index(inplace = True) # sordime indexi järgi
    else:
        df.sort_values(by=[aastad[-1] + ' %'], inplace=True, ascending=False) # sordime viimase aasta järgi
    return df, tulba_nimi
    
def plot_pie(df, tiitel):
    dff = df.drop(aastad, axis=1, inplace=False)
    for index, col in enumerate(dff.columns):
        ax = df[col].plot(kind='pie', autopct = '%1.1f%%', legend=False, figsize=(5, 5), title=aastad[index]+" : "+tiitel)
        ax.set(ylabel = '')
        plt.show()

In [ ]:
# START - COL 3 -'Kuidas hindad oma programmeerimisoskusi / teadmisi enne selle kursuse algust?'

df, tiitel = col_stat(df_start, 3)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')
    
# tulemused sarnased, vaid väga tugevate osakaal on väheke kasvanud
# võib eeldada, et umbes 5% oleks võimelised eeleksamit tegema

# Samad andmed sektordiagrammiga, kus vastanute hulk on 100% - see annab parema pildi
plot_pie(df, tiitel)

In [ ]:
# Eelnevalt sai testitud ka lineaarse regressiooni mudelit, kus andmed sai teisendatud one-hot vektoriteks
# Lineaarse regressiooni mudel punktide järgi väga head täpsust ei andnud
# Täpsus oleks olnud veidike parem, kui oleks ennustanud lõpphinnet A, B, C jne.,
# kuid ka siis ei oleks see väga täpne olnud, sest hinded jagunevad eri tasemetel suhteliselt ühtlaselt.

# Punktide jaotus vastavalt eelnevale oskusele
sort_tase = ['Olen suht kõva käsi, arvan, et siin kursusel ma väga palju juurde ei õpi',
            'Oskan lihtsaid programme luua (võibolla ka mingis muus keeles kui Python)',
            'Olen varem proovinud pisut programmeerida, aga pole kaugele jõudnud',
            'Mingi ettekujutus programmeerimisest on, aga ise polnud proovinud',
            'Ei tea programmeerimisest peaaegu midagi']

def plot_scatter(punktitulp = 'Hinne (Punktid)', tiitel = 'Lõpp-', nulliga_lõpetanud = True):
    for aasta in aastad:
        if nulliga_lõpetanud: # kas nulliga lõptatnud jätta andmete hulka
            df = df_start[aasta].loc[:, [df_start[aasta].columns[3], punktitulp]].copy()
        else: # või ei
            df = df_start[aasta][df_start[aasta]['Hinne (Punktid)'] != 0].loc[:, [df_start[aasta].columns[3], punktitulp]].copy()
        
#         display(df[df.columns[0]].unique()) # nimetused sortimiseks
        df[df.columns[0]] = pd.Categorical(df[df.columns[0]], sort_tase[::-1])
        df.sort_values(by=df.columns[0], inplace=True) # vajalik, et kõik tabelid tuleks sama Y telje järjestusega
        plt.figure(figsize=(8,2))
        plt.scatter(df[df.columns[1]], df[df.columns[0]])
        plt.title(aasta + ' : ' + tiitel + 'punktide jaotus vastavalt eelnevale kogemusele')
        plt.show()
        
        if nulliga_lõpetanud: # kui nulliga lõptanud sees, siis zoomi ilma 0 hindeta
            df2 = df[df[df.columns[1]] > 0].copy()
#             df2.sort_values(by=df2.columns[0], inplace=True)
            plt.figure(figsize=(8,2))
            plt.scatter(df2[df2.columns[1]], df2[df2.columns[0]])
            plt.title(aasta + ' : ' + tiitel + 'punktide jaotus vastavalt eelnevale kogemusele, punktid > 0 ')
            plt.show()

plot_scatter('Hinne (Punktid)', 'Lõpp-', True)

In [ ]:
# START - COL 4 - 'Täpsustus: kui oled programmeerimisega kokku puutunud, siis kuidas?'

# Siin paljud on vastanud mitu valikut

df, tiitel = col_stat(df_start, 4, True)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# START - COL 5 - Oleta, millistes rollides võiksid sa tulevikus oma programmeerimisoskust otseselt või kaudselt rakendada. Vali need, mis tunduvad sulle enam-vähem realistlikud.

df, tiitel = col_stat(df_start, 5, True)
tiitel = 'Eeldatavad töörollid tulevikus'
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest', (3,6))
# plot_pie(df, tiitel)

In [ ]:
# START - COL 6 - Mida see programm võiks teha ja miks sa nii arvad? Tegemist on väljamõeldud programmeerimiskeelega, seetõttu pole siin ei õiget ega valet vastust.

# Ülesanne: Uuri järgnevat programmi:

# let
#   x = 2
#   y = x + z
#   z = 3
# in
#   show y

# 2018 andmete põhjal excelis käsitsi grupeeritud (2019 peaks statistiliselt sarnane olema)

col6 = [['5', 27.4], ['y või y tulemus', 26.2], ['error', 23.9], ['arvutab midagi', 9.4], ['vastus puudub', 7.1], ['2 + z', 2.3], ['2', 1.4], ['ei tea', 0.6], ['2 + 3', 0.6], ['x + z', 0.6], ['2z', 0.3], ['x + y', 0.3]]
df = pd.DataFrame(col6, columns = ['vastus', 'vastanute %'])
df.set_index('vastus', inplace = True)
ax = df.plot(kind = 'barh', figsize = (8,4))
ax.xaxis.set_major_formatter(mtick.PercentFormatter(decimals = 0))
ax.invert_yaxis()
ax.legend(loc='lower right')
plt.title(df_start[aastad[0]].columns[6].split('?')[0] + '?')
plt.show()
display(df)

In [ ]:
# START - COL 7 - Mida sa kursuselt ootad? Kas arvad, et saab olema kerge? Raske? Igav? Huvitav? Miks? Midagi muud, mida soovid meile öelda? Siin saad ka täpsustada või kommenteerida eelmiste küsimuste vastuseid.

# võtame tekstist mitteolulised märgid välja
def puhasta_teksti_tulp(df, tulp):
    data = ""
    for item in df[tulp]:
        if item != 'Andmed puuduvad' and isinstance(item, str) and len(item) > 1:
            text = item.lower()
            text = re.sub(':|;|\(|\)|-|!|\?|,|\.|\*|&quot;', '\n', text)
            text = re.split('\n', text)
            data += " ".join(text) + " "
            data = re.sub(' +', ' ', data)
    return data.strip()

# koostame sõnapilve, kus on eemaldatud stopsõnad
def sõnapilv(text, stops=""):
    # stopwords = set(STOPWORDS)
    # stopwords.update(["et", "ja", "ka", "kuid", "see", "aga", "ning", "siis", "kui", "on", "prog", "gramm"])
    stopwords = set(["et", "ja", "ka", "kuid", "see", "aga", "sest", "ning", "siis", "kui", "on"] + stops.split())
    wordcloud = WordCloud(stopwords=stopwords).generate(text) # background_color="white"
    plt.figure(figsize=[10,10])
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    
# Puhastame - otsime unikaalsed sõnad ja nende sagedused
def top_sõnad(tekst):
    sõnad = tekst.split()
    freq = {}
    
    for sõna in sõnad:
        if len(sõna) > 3: # vaatame vaid üle 3 tähelisi sõnu
            if sõna in freq.keys():
                freq[sõna] += 1
            else:
                võti = [k for k, v in freq.items() if k.startswith(sõna[:3])] # kui key-de hulgas on sõna, mille 3 esimest tähte on samad
                if len(võti) > 0:
                    freq[võti[0]] += 1 # siis loeme samaks sõnaks
                else:
                    freq[sõna] = 1

    # teeme uue df-i sõnadest ja sagedustest
    df = pd.DataFrame(freq.items(), columns=['sõna', 'sagedus'])
    df.sort_values(by=['sagedus'], inplace=True, ascending=False)
    df.reset_index(inplace=True)
    df = df.drop('index', 1)
    display(df.head(10)) # väljastame 10 esimest top sõna
    
    # Genereerime uue teksti sõnadest vastavalt sagedusele
    text = ""
    for index, row in df.iterrows():
        text += (row['sõna'] + " ") * row['sagedus']
    data = text.split()
    random.shuffle(data)
    return " ".join(data)

# sõnapilv kirjavahemärkidest puhastatud tekstist
def koosta_sõnapilv(dfs, tulp, topsõnad = False, stops="", väljasta_text = False):
    tulba_nimi = dfs[aastad[0]].columns[tulp]
    
    for aasta in aastad:
        text = puhasta_teksti_tulp(dfs[aasta], tulba_nimi)
        print(aasta, '(puhastamata):', tulba_nimi)
        
        sõnapilv(text, stops)
        
        if väljasta_text:
            for item in list(dfs[aasta][tulba_nimi]):
                if item != 'Andmed puuduvad':
                    print(item)
        
        if topsõnad: # sõnapilv eraldi puhastatud tekstist, kus vaadatakse vaid top sõnu ja sarnased sõnad loetakse samaks
            print('\n', aasta, '(top sõnad):', tulba_nimi)
            text = top_sõnad(text)
            sõnapilv(text, stops)

In [ ]:
# Sõnapilved ei anna täpset vastust, kuid annavad märksõnad, millele tuleks rohkem rõhku panna ja mida
# võiks vastuste seast otsida ja lähemalt uurida
# Iga sõnapilve juures tuleb lõplikus töös ka väljavõte peamistest probleemidest ja tähelepanekutest
# Samuti üritab leida uusi ideid või häid soovitusi
# Suurte tekstivastuste hulga analüüs (kvalitatiivne) on peamiselt käsitsi parimate vastuste üles leidmine ja esile toomine
# Lisan need vaid lõppdokumenti käsitsi.

koosta_sõnapilv(df_start, 7, True)

# OSA 3

# IGANÄDALASED KÜSIMUSTIKUD

In [ ]:
nädalad = lehed[1:-2] # nädalaküsitlused (Start, Lõpp ja Hinded jäävad välja)

sort_lihtsus = ['Väga lihtne', 'Lihtne', 'Pigem lihtne', 'Pigem keeruline', 'Keeruline', 'Väga keeruline', 'Andmed puuduvad']
sort_aeg = ['alla 1 tunni', '1-2 tundi', '2-4 tundi', '4-8 tundi', '8-16 tundi', 'üle 8 tunni', 'üle 16 tunni', 'Andmed puuduvad']
sort_aeg2 = ['alla 1 tunni', '1-2 tundi', '2-4 tundi', '4-8 tundi', '8-16 tundi', 'üle 16 tunni', 'Andmed puuduvad']

def loe_andmed_nädal(): # loeme mitme aasta nädalaküsimustikud sisse
    dfs = {}
    size = {}
    for aasta in aastad:
        küsimustikud = []
        suurused = []
        xlsx = pd.ExcelFile(aasta + '.xlsx')
        for nädal in nädalad:
            df = pd.read_excel(xlsx, nädal)
            df = df.fillna("Andmed puuduvad")
            df = df.replace(to_replace = '–', value = '-', regex = True) # eri aastatel eri märk
            küsimustikud.append(df)
            suurused.append(küsimustikud[-1].shape[0]) # df-i ridade arv
        dfs[aasta] = küsimustikud
        size[aasta] = suurused
    return dfs, size

df_nädal, size_nädal = loe_andmed_nädal()

In [ ]:
# Nädalaküsimustikke andmed
print('Nädalaküsimustikke kokku:', len(nädalad))
print('Nädalaküsimustikkude nimed:', nädalad)

print('Nädalaküsimustikkudele vastanute arvud:')
display(size_nädal)
   
print('Vastanute arv protsentides stardiküsitluse koguarvu suhtes:')
df_nädal_prot = df_protsendid.drop(['Start', 'Lõpp'], axis=1) 
display(df_nädal_prot)
plot_protsendid(df_nädal_prot)

### Iganädalased küsimustikud

In [ ]:
# Nädalaküsitlustes (2-15) on iga nädal täpselt samad küsimused: 
# COL 3: Kuidas hindad enda edasijõudmist aines?
# COL 4: Millistest üritustest sel nädalal osa võtsid?
# COL 5: Kui palju aega kulutasid kodus selle nädala teemadele (sh materjali lugemine ja koduülesannete lahendamine)?
# COL 6: Kui keeruline oli sinu jaoks selle nädala kodutöö?
# COL 7: Üldine tagasiside (nt. mida võiks järgmises loengus või praktikumis käsitleda või teisiti teha)

# KT1 puhul on alates COL5-st erinevad küsimused:
# COL 5: Kui palju aega kulutasid kodus vaheeksamiks ettevalmistamisele (sh materjali lugemine ja harjutusülesannete lahendamine)?
# COL 6: Kui keeruline oli vaheeksam sinu jaoks?
# COL 7: Kuidas suhtud automaatkontrolli? Kas eelmise selleteemalise küsitlusega võrreldes on midagi muutunud?
# COL 8: Üldine tagasiside (nt. mida võiks järgmises loengus või praktikumis käsitleda või teisiti teha)

# KT2 puhul on alates COL5-st erinevad küsimused:
# COL 5: Kui palju aega kulutasid kodus kontrolltööks ettevalmistamisele (sh materjali lugemine ja harjutusülesannete lahendamine)?
# COL 6: Kui keeruline oli sinu jaoks kontrolltöö test?
# COL 7: Kui keerulised olid sinu jaoks kontrolltöö programmeerimisülesanded?
# COL 8: Millistel teemadel tunned ennast ebakindlalt?
# COL 9: Kuidas tavaliselt käitud, kui koduülesande lahendamisel tekib tõrge?
# COL10: Kuidas hindad automaatkontrolli kasulikkust?
# COL11: Muud kommentaarid automaatkontrolli kohta
# COL12: Üldine tagasiside (nt. mida võiks järgmises loengus või praktikumis käsitleda või teisiti teha)

# Leiab tulbast unikaalsed kirjed, mitme vastuse variandi puhul on võimalik nende eraldamine ja sulgudes andmete kõrvaldamine
def leia_unikaalsed(dfs, tulp, eraldi = False, sulgudesinfo = True):
    unik = set()
    for df in dfs:
        for info in df[df.columns[tulp]].unique():
            if not sulgudesinfo: # sulgudes info jätame välja
                info = re.sub('\(.*\)', '', info).strip()
                
            if eraldi: # kui vastuse variante on mitu
                for item in info.split('\n'): 
                    unik.add(item)
            else:
                unik.add(info.replace('\n', '+')) # kui soovime andmeid koos vaadata
    unik = list(unik)
    unik.sort()
    return unik

def loo_olekute_loendi_tabel(dfs, tulp, unikaalsed, eraldi = False):
    read = []
    for index, df in enumerate(dfs):
        sõnastik = {}
        info = []
        for rida in df[df.columns[tulp]]:
            if eraldi:
                for item in rida.split('\n'):
                    info.append(item)
            else:
                info.append(rida.replace('\n', '+'))

            for unik in unikaalsed:
                sõnastik[unik] = info.count(unik)
        sõnastik['Kokku'] = df.shape[0] # len(info) - len mitme valikuga vastuse puhul annab vastsute koguarvu, kuid see pole hea võrdlus
        read.append(sõnastik)
    
    return pd.DataFrame(read)

def arvuta_tabel_protsentideks(tabel):
    for col in tabel.columns[:-1]: # Kokku tulpa ei arvuta
        tabel[col] = round(tabel[col] / tabel['Kokku'] * 100, 1) # arvutame andmed ümber protsentideks
    return tabel

# tulp, graafiku tüüp, mitu valikut eraldada, sulgudes info kõrvaldada, stacked bar, sortimise järjestus, legend pildilt välja
def tabel_nädal(tulp, graafik = 'line', eralda = False, sulgudesinfo = True, stack = False, järjestus = '', legout = False):
    for aasta in aastad:
        küsimus = df_nädal[aasta][0].columns[tulp]
        unik = leia_unikaalsed(df_nädal[aasta], tulp, eralda, sulgudesinfo)
        tabel = loo_olekute_loendi_tabel(df_nädal[aasta], tulp, unik, eralda)
        tabel.index = nädalad
        
        if len(tabel.columns) == len(järjestus):
            tabel = tabel.reindex(columns = järjestus)
        elif len(järjestus) > 0:
            print('Sortimise viga')
        
        print(aasta, '(absoluutarvud):', küsimus)
        display(tabel)
        tabel = arvuta_tabel_protsentideks(tabel)
        print(aasta, '(protsendid):', küsimus)
        display(tabel)
        plot_nädal(aasta + ': ' + küsimus, tabel, graafik, stack, legout)
        
def plot_nädal(pealkiri, tabel, graafik = 'line', stack = False, legout = False):
    tabel = tabel.reset_index(drop = True, inplace = False)
    
    if graafik == 'bar' and not stack:
        ax = tabel.iloc[:,0:-1].plot(kind = graafik, figsize = (10,5), stacked = stack, width = 1) # Kokku tulpa ei näita
    else:
        ax = tabel.iloc[:,0:-1].plot(kind = graafik, figsize = (10,5), stacked = stack) # Kokku tulpa ei näita
    handles, labels = ax.get_legend_handles_labels()
    
    if legout: # legend pildilt välja
        plt.legend(reversed(handles), reversed(labels), loc='center left', fancybox=True, framealpha=0.5, bbox_to_anchor=(1, 0.5))
    else:
        plt.legend(tabel.columns[:-1], loc='best', fancybox=True, framealpha=0.5) # parimal positsioonil
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals = 0))
    plt.ylabel('vastanute % kõigist vastusest')
    plt.xlabel('nädala number')
    plt.title(pealkiri)
    plt.xticks(range(len(nädalad)), labels = nädalad)
    plt.show()

In [ ]:
# N2-15 - COL 3 - Kuidas hindad enda edasijõudmist aines?
tabel_nädal(3)

In [ ]:
# # Leiab kõigi kodutööde küsitluste vastanute nimed
# def vastanute_nimed(dfs):
#     nimed = []
#     for df in dfs:
#         nimed += list(df[df.columns[0]])
#     return set(nimed)

# def loo_staatuse_muutuse_tabel(dfs, tulp, unik):
#     nimed = vastanute_nimed(dfs)
#     asendused = {unik[i] : i for i in range(0, len(unik))}
#     read = []
# #     test = pd.DataFrame(dict.fromkeys(unik, []))

#     for nimi in nimed:
#         rida = [nimi] # V2 [nimi]
#         for df in dfs:
#             if nimi in list(df[df.columns[0]]): # kui õpilase andmed on selle kodutöö kohta olemas
#                 loc = df.loc[df[df.columns[0]] == nimi].index[0]
#                 staatus = asendused[df.iloc[loc][3]]
#                 if staatus == 0 and len(rida) > 1: # andmed puuduvad
#                     staatus = rida[-1] # viimane seis
#                 rida.append(staatus)
#             else:
#                 if len(rida) == 1:
#                     rida.append(0)
#                 else:
#                     rida.append(rida[-1]) # lisame eelmise seisu
#         read.append(rida)
        
#     uusdf = pd.DataFrame(read, columns = ["Nimi"] + kodutööd)
#     uusdf.set_index("Nimi", inplace=True)
#     return uusdf

In [ ]:
# JÄÄB AJAPUUDUSEL VÄLJA, KUNA SIIN OLEKS VAJA TÄIESTI UUS GRAAFIKU TÜÜP LEIUTADA, ET SEE LOETAV OLEKS

# 2018 - kuidas käib ühest staatusest teise liikumine järgnevate kodutöödega
# sisuliselt 3D maatriksi kujutamine 2D kujul; 3D (x,y,z) - kodutdöö_nr, edasijõudmine, kursusel osaleja
# eeldus, et kui järgmine kord pole vastanud, siis on samas seisus, mis eelmine kord
# sellele küsimusele peaks vastama iga kodutöö esitamise juures või iga loeng
# seda võiks veel täiustada kaaludega, kus joone paksus sõltuks kaalust

# unik = leia_unikaalsed(dfs8, 3, False)
# df8 = loo_staatuse_muutuse_tabel(dfs8, 3, unik)
# df8 = df8.T # transponeeritud tabel
# df8.reset_index(inplace=True)
# df8 = df8.drop('index', 1)

# plt.figure(figsize=(10,5))
# plt.plot(df8)
# plt.ylabel('')
# plt.xlabel('kodutöö number')
# plt.title(aasta8 + ' muutused edasijõudmise olekus')
# plt.xticks(range(len(kodutööd)), labels = kodutööd)
# plt.yticks(range(len(unik)), labels = unik)
# plt.show()

# # 2019 - kuidas käib ühest staatusest teise liikumine järgnevate kodutöödega

# unik = leia_unikaalsed(dfs9, 3, False)
# df9 = loo_staatuse_muutuse_tabel(dfs9, 3, unik)
# df9 = df9.T # transponeeritud tabel
# df9.reset_index(inplace=True)
# df9 = df9.drop('index', 1)

# plt.figure(figsize=(10,5))
# plt.plot(df9)
# plt.ylabel('')
# plt.xlabel('kodutöö number')
# plt.title(aasta9 + ' muutused edasijõudmise olekus')
# plt.xticks(range(len(kodutööd)), labels = kodutööd)
# plt.yticks(range(len(unik)), labels = unik)
# plt.show()

In [ ]:
# N2-15 - COL 4 - Millistest üritustest sel nädalal osa võtsid/võtad?
# tabel_nädal(4) # andmed eraldamata - see liialt sigri-migri

tabel_nädal(4, 'bar', True, True, False, '', True) # Andmed eraldatud, st Loengud, Praktikumid jms loetakse eraldi

In [ ]:
# N2-15 - COL 5 - Kui palju aega kulutasid kodus selle nädala teemadele (sh materjali lugemine ja koduülesannete lahendamine)?
# tabel_nädal(5, 'line', False, False)
# tabel_nädal(5, 'bar', False, False)
tabel_nädal(5, 'bar', False, False, True, sort_aeg + ['Kokku'], True)

### Nädalaküsitluste COL 5-7 ilma kontrolltööde tagasisideta

In [ ]:
# Eemaldame KT-d nädalaküsitlustest ja paneme need eraldi df-i, kuna siin lähevad küsitlused erinevaks
if len(df_nädal[aastad[0]]) == len(lehed) - 3: # et topelt ei kustutaks, mitmekordsel käivitamisel
    df_kt1 = {}
    df_kt2 = {}
    KT1 = nädalad.index('KT1')
    KT2 = nädalad.index('KT2')
    nädalad = np.delete(nädalad, [KT1, KT2]).tolist()

    for aasta in aastad:
        df_kt1[aasta] = df_nädal[aasta].pop(KT1)
        df_kt2[aasta] = df_nädal[aasta].pop(KT2 - 1) # -1, kuna eelnevalt on pop tehtud

In [ ]:
# N2-15 - COL 6 - Kui keeruline oli sinu jaoks selle nädala kodutöö?
tabel_nädal(6, 'bar', False, False, True, sort_lihtsus + ['Kokku'], True)

In [ ]:
def puhasta_teksti_loend(loend): # siia saada list
    data = ""
    for item in loend:
        if item != 'Andmed puuduvad' and len(item) > 1:
            if isinstance(item, list):
                data += " " + puhasta_teksti_loend(item) # rekursioon
            else:
                text = item.lower()
                text = re.sub(':|;|\(|\)|-|!|\?|,|\.|\*|&quot;', '\n', text)
                text = re.split('\n', text)
                data += " ".join(text) + " "
            data = re.sub(' +', ' ', data)
    return data.strip()

def leia_tagasiside(dfs, tulp):
    tekst = []
    kogus = []
    if isinstance(dfs, pd.DataFrame):
        tekst = list(filter(lambda a: a != "Andmed puuduvad" and len(a) > 1, dfs[dfs.columns[tulp]]))
        kogus.append(len(tekst))
    else:
        for df in dfs:
            read = list(filter(lambda a: a != "Andmed puuduvad" and len(a) > 1, df[df.columns[tulp]]))
            kogus.append(len(read))
            tekst.append(read)
    
    return tekst, kogus

def tagasiside(dfs, tulp, topsõnad = False):
    tulba_nimi = dfs[aastad[0]][0].columns[tulp]
    
    for aasta in aastad:
        tekst, kogus = leia_tagasiside(dfs[aasta], tulp)
        print(aasta, ':', tulba_nimi, '\n')
        print(aasta, 'tagasiside kogus:', kogus, '\n')
        for index, sisu in enumerate(tekst):
            print(aasta, ': nädal', nädalad[index], '(' + str(kogus[index]) + '): ', sisu)
        text = puhasta_teksti_loend(tekst)
        sõnapilv(text)

        if topsõnad: # sõnapilv eraldi puhastatud tekstist, kus vaadatakse vaid top sõnu ja sarnased sõnad loetakse samaks
            print(aasta, '(top sõnad):', tulba_nimi)
            text = top_sõnad(text)
            sõnapilv(text)

In [ ]:
# N2-15 - COL 7 - Üldine tagasiside (nt. mida võiks järgmises loengus või praktikumis käsitleda või teisiti teha)
# Kirjalikku töösse lisan ka kokkuvõtte, mida rohkem esile tuuakse ja millele võiks eraldi tähelepanu pöörata
# Kuna tagasiside andnute arv on kursusel osalejate arvust väga väike osa, siis vastused on väga seinast seina:
# kes on 100% rahul kuni suure kurtmiseni, et väga raske või aega pole
# Praktikumid: tempo liialt kiire, paaristöö ei sobi
# Kodutööd: paljudel võtab kodutöö liialt aega
# Loengud: mitmeid häirib arvutite kasutamine (mängimine jms) loengu ajal

tagasiside(df_nädal, 7, True)

# KONTROLLTÖÖ 1 COL 5-12

In [ ]:
# 2018 ja 2019 tulpade pealkirjad sai samaks pandud, kuna neil sisu on sama, kuid pealkirja tekstid veidike erinevad
df_kt1['2018'].columns = df_kt1['2019'].columns
for aasta in aastad:
    df_kt1[aasta] = df_kt1[aasta].replace(to_replace = '–', value = '-', regex = True) # eri aastatel eri märk

In [ ]:
# KT1 - COL 5 - Kui palju aega kulutasid kodus kontrolltööks ettevalmistamisele (sh materjali lugemine ja harjutusülesannete lahendamine)?
df, tiitel = col_stat(df_kt1, 5, False)
tiitel = tiitel.split('(')[0][:-1] + '?'
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest', (8, 4), True, sort_aeg)

In [ ]:
# KT1 - COL 6 - Kui keeruline oli sinu jaoks kontrolltöö test?
df, tiitel = col_stat(df_kt1, 6, False)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest', (8, 4), True, sort_lihtsus)

In [ ]:
# KT1 - COL 7 - Kui keeruline olid sinu jaoks kontrolltöö programmeerimisülesannete osa?
df, tiitel = col_stat(df_kt1, 7, False)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest',(8, 4) , True, sort_lihtsus)

In [ ]:
# KT1 - COL 8 - Millistel teemadel tunned ennast ebakindlalt?
# selles tulbas oli mitmeid muudatusi eri aastate vahel
df_kt1['2018'] = df_kt1['2018'].replace(to_replace = 'vms\.\)', value = 'vms)', regex = True)
df_kt1['2018'] = df_kt1['2018'].replace(to_replace = 'Olen ebakindel põhiliste arvutehetega', value = 'Olen ebakindel põhiliste arvutehete juures', regex = True)
df_kt1['2018'] = df_kt1['2018'].replace(to_replace = 'Olen ebakindel põhiliste sõnetehetega', value = 'Olen ebakindel põhiliste sõnetehete juures', regex = True)
df_kt1['2018'] = df_kt1['2018'].replace(to_replace = 'Programmeerimine tundub mulle üldiselt müstiline ja hirmutav', value = 'Programmeerimine tundub mulle üldse müstiline ja hirmutav', regex = True)
df_kt1['2018'] = df_kt1['2018'].replace(to_replace = 'Avaldiste ja lausete olemust on veel segane', value = 'Avaldiste ja lausete olemus on veel segane', regex = True)

df, tiitel = col_stat(df_kt1, 8, True)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest', (4, 4))

In [ ]:
# KT1 - COL 9 - Kuidas tavaliselt käitud, kui koduülesande lahendamisel tekib tõrge?
df_kt1['2018'] = df_kt1['2018'].replace(to_replace = 'Küsin abi sõbralt', value = 'Küsin abi sõbralt või kursusekaaslastelt', regex = True)

df, tiitel = col_stat(df_kt1, 9, True)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# KT1 - COL 10 - Kuidas hindad automaatkontrolli kasulikkust?
df_kt1['2018'] = df_kt1['2018'].replace(to_replace = 'Automaatkontroll oma “tähenärimisega” pigem segab', value = 'Automaatkontroll oma „tähenärimisega” pigem segab', regex = True)

df, tiitel = col_stat(df_kt1, 10, True)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# KT1 - COL 11 - Muud kommentaarid automaatkontrolli kohta
koosta_sõnapilv(df_kt1, 11, True, 'automaatkontroll automaatkontrolli automaatkontrollist', True)

In [ ]:
# KT1 - COL 12 - Üldine tagasiside (nt. mida võiks järgmises loengus või praktikumis käsitleda või teisiti teha)
koosta_sõnapilv(df_kt1, 12, True, '', True)

### KONTROLLTÖÖ 2 COL 5-8

In [ ]:
# 2018 ja 2019 tulpade pealkirjad sai samaks pandud, kuna neil sisu on sama, kuid pealkirja tekstid veidike erinevad
df_kt2['2018'].columns = df_kt2['2019'].columns # ühtlustame küsimused, kuna seal sõnade järjestus kohati erinev
for aasta in aastad:
    df_kt2[aasta] = df_kt2[aasta].replace(to_replace = '–', value = '-', regex = True) # eri aastatel eri märk
    df_kt2[aasta] = df_kt2[aasta].replace(to_replace = 'ülle', value = 'üle', regex = True) # ühes aastas typo

In [ ]:
# KT2 - COL 5 - Kui palju aega kulutasid kodus kontrolltööks ettevalmistamisele (sh materjali lugemine ja harjutusülesannete lahendamine)?
df, tiitel = col_stat(df_kt2, 5, False)
tiitel = tiitel.split('(')[0][:-1] + '?'
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest', (8, 4), True, sort_aeg2[:-1])

In [ ]:
# KT2 - COL 6 - Kui keeruline oli sinu jaoks kontrolltöö?
df, tiitel = col_stat(df_kt2, 6, False)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest', (8, 4), True, sort_lihtsus[:-1])

In [ ]:
# KT2 - COL 7 - Kuidas suhtud automaatkontrolli? Kas su suhtumine on kursuse jooksul muutunud?
koosta_sõnapilv(df_kt2, 7, True, 'automaatkontroll', True)

In [ ]:
# KT2 - COL 8 - Üldine tagasiside (nt mida võiks järgmises loengus või praktikumis käsitleda või teisiti teha)
koosta_sõnapilv(df_kt2, 8, True, '', True)

# OSA 4

# LÕPUKÜSITLUS

In [ ]:
# COL	3	Tooge välja õppetegevus, mis teie õppimist kõige rohkem toetas. Miks te seda kõige toetavamaks pidasite?
# COL	4	Tooge välja õppetegevus, mis teie õppimist kõige vähem toetas. Selgitage, mis võiks selle juures olla teisiti, et toetus oleks suurem.
# COL	5	Igal nädalal oli 2 tundi loenguid teisipäeviti ja 2 tundi praktikume neljapäeviti ning lisaks konsultatsioon kolmapäeviti. Millised on teie ettepanekud selle struktuuri muutmise kohta?
# COL	6	Palun kirjutage kommentaare kontrolltööde kohta.
# COL	7	Palun kirjutage kommentaare eksamile pääsemise tingimuste kohta.
# COL	8	Palun kirjutage kommentaare eksami kohta.
# COL	9	Mil viisil enamasti osalesite loengutes semestri alguses (1.-6. nädal)?
# COL	10	Mil viisil enamasti osalesite loengutes semestri keskel (7.-12. nädal)?
# COL	11	Mil viisil enamasti osalesite loengutes semestri lõpus (13.-16. nädal)?
# COL	12	Miks te just nii käitusite? Kui semestri jooksul viisi muutsite, siis miks?
# COL	13	Loeng on tänapäeval programmeerimise kursusel mõttetu.
# COL	14	Loeng oleks kasulikum kui seda korraldataks teistmoodi.
# COL	15	Soovi korral kirjutage kommentaare loengu kohta.
# COL	16	Käisin loengus või vaatasin loengu videot põhiliselt selleks, et punkte saada.
# COL	17	Käiksin loengus või vaataksin loengu videot ka siis, kui selle eest üldse punkte ei antaks.
# COL	18	Käiksin loengus või vaataksin loengu videot tihedamalt, kui punkte antaks rohkem.
# COL	19	Tegin loengus (või loengu videot vaadates) märkmeid.
# COL	20	Soovi korral täpsustage, kuidas tegite märkmeid loengus või loengu videot vaadates.
# COL	21	Oleksin rohkem märkmeid teinud, kui nutiseadmed (telefonid, arvutid) oleksid olnud lubatud
# COL	22	Oleksin  nutiseadmeid (telefone, arvuteid) kasutanud asjakohaselt, kui kasutamine oleks lubatud.
# COL	23	Oleksin nutiseadmeid (telefone, arvuteid) kasutanud kõrvalisteks tegevusteks, kui kasutamine oleks lubatud.
# COL	24	Paljud üliõpilased loengus siiski kasutasid nutiseadmeid (telefone, arvuteid).
# COL	25	Soovi korral kommenteerige nutiseadmete (telefonide, arvutite) lubamist/keelamist.
# COL	26	Klikkerite kasutamine oli asjakohane.
# COL	27	Kllikkerite kasutamine aitas vältida tähelepanu hajumist.
# COL	28	Klikkerite kasutamine aitas tõsta huvi loengus võetud teema vastu.
# COL	29	Klikkerid aitasid kaasa kriitilise mõtlemise arendamisel.
# COL	30	Klikkerite kasutus julgustas loengutes osalema.
# COL	31	Loengu videosse võiks olla integreeritud küsimusi, millele saab interaktiivselt vastata.
# COL	32	Soovi korral kommenteerige klikkerite kasutamist.
# COL	33	Palun lisage veel kommentaare, mis aitaksid kursust arendada.

In [ ]:
# Loeme sisse kaks lehte ja liidame kokku üheks DF-ks
# ei loeta neid, kellel puudub rühm ja seega ka hilisemad andmed
# sai kontrollitud, et ei oleks mitut sama nimega

df_lõpp = loe_andmed('Lõpp')

osalejad = []
for aasta in aastad:
    start = df_start[aasta].shape[0]
    lõpp = df_lõpp[aasta].shape[0]
    p0 = df_startstat[aasta]['Punktid > 0']
    p50 = df_startstat[aasta]['Punktid $\geq$ 50']
    osalejad.append([start, lõpp, round(lõpp / start * 100, 1), p0, p50, round(p50 / start * 100, 1)])
    print (aasta, 'lõpuküsitlusele vastanuid:', lõpp)

In [ ]:
# Üldine - alguse ja lõpu võrdlus, lõpu osalus stardi suhtes, lõpuni jõudjad (punktid >= 50) ning pos. lõpetajate % stardi suhtes
df_kokkuvõte = pd.DataFrame(osalejad, index = aastad, columns = ['stardiküsitlus', 'lõpuküsitlus', 'lõpp/start %', 'punktid > 0', 'punktid $\geq$ 50', 'p50/start %'])
display(df_kokkuvõte)

In [ ]:
# lõpuküsitluse ja positiivsel lõpetanute protsent stardiküsitluse koguarvu (100%) suhtes
df = df_kokkuvõte[['lõpuküsitlus', 'punktid $\geq$ 50']].copy()
for col in df.columns:
    df[col] = round(df[col] / df_kokkuvõte['stardiküsitlus'] * 100, 1) # arvutame andmed ümber protsentideks
plot_barh_perc(df.T, 'Positiivse hindega lõpetanud ja lõpuküsitlusele vastanud', 'stardiküsitluses osalenute koguarvu suhtes', (8,2), False)

In [ ]:
# Lõpp - COL 3 - Tooge välja õppetegevus, mis teie õppimist kõige rohkem toetas. Miks te seda kõige toetavamaks pidasite?
koosta_sõnapilv(df_lõpp, 3, True, '', True)

In [ ]:
# Lõpp - COL 4 - Tooge välja õppetegevus, mis teie õppimist kõige vähem toetas. Selgitage, mis võiks selle juures olla teisiti, et toetus oleks suurem.
# koosta_sõnapilv(df_lõpp, 4, True, 'üles praktikum loeng loengu enda sest väga minu loengud praktikumi')
koosta_sõnapilv(df_lõpp, 4, True, '', True)

In [ ]:
# Lõpp - COL 5 - Igal nädalal oli 2 tundi loenguid teisipäeviti ja 2 tundi praktikume neljapäeviti ning lisaks konsultatsioon kolmapäeviti. Millised on teie ettepanekud selle struktuuri muutmise kohta?
# koosta_sõnapilv(df_lõpp, 5, True, 'loeng praktikum struktuur loengu praktikumi struktuur võiks olla enda võiks konsultatsioon oleks minu üles')
koosta_sõnapilv(df_lõpp, 5, True, '', True)

In [ ]:
# Lõpp - COL 6 - Palun kirjutage kommentaare kontrolltööde kohta.
# koosta_sõnapilv(df_lõpp, 6, True, 'roll kontroll kontrolltöö arvestuslik tööd kontrolltööd olid punkti olid ülesarvestusliku ülesande kontrolltööde')
koosta_sõnapilv(df_lõpp, 6, True, '', True)

In [ ]:
# Lõpp - COL 7 - Palun kirjutage kommentaare eksamile pääsemise tingimuste kohta.
# koosta_sõnapilv(df_lõpp, 7, True, 'tingimus tingimuse eksam tingimused eksami eksamil pääs eksamile pääsemis pääsemise olid')
koosta_sõnapilv(df_lõpp, 7, True, '', True)

In [ ]:
# Lõpp - COL 8 - Palun kirjutage kommentaare eksami kohta.
# koosta_sõnapilv(df_lõpp, 8, True, 'eksam üles eksami arvestus arvestuslik punkti roll arvestusliku ülesande väga test ülesanne')
koosta_sõnapilv(df_lõpp, 8, True, '', True)

In [ ]:
def tabel_abs_prot(tulp, mitu_tulpa_koos = 1, nimed = '', pealkiri = ''):
    for aasta in aastad:
        küsimus = df_lõpp[aasta].columns[tulp]
        ser = []

        for i in range(mitu_tulpa_koos):
            ser.append(df_lõpp[aasta][df_lõpp[aasta].columns[tulp + i]].value_counts())
        df = pd.DataFrame(ser)
        df['Kokku'] = df.sum(axis = 1)
        
        if len(df.index) == len(nimed):
            df = df.set_index([pd.Index(nimed)])
        
        print(aasta, '(absoluutarvud):', küsimus)
        display(df)
        df = arvuta_tabel_protsentideks(df)
        print(aasta, '(protsendid):', küsimus)        
        display(df)
        plot_prot(df, aasta, pealkiri)
        
def plot_prot(tabel, aasta, pealkiri = 'Küsimustikele tagasiside andnute arv protsentides'):
    ax = tabel.iloc[:,0:-1].plot(kind = 'barh', figsize=(8,2), stacked = True) # Kokku tulpa ei näita
    ax.invert_yaxis() # õigetpidi järjestus
    plt.legend(tabel.columns[:-1], loc='center left', fancybox=True, framealpha=0.5, bbox_to_anchor=(1, 0.5))
    ax.xaxis.set_major_formatter(mtick.PercentFormatter(decimals = 0))
    plt.xlabel('vastanute % kõigist vastusest')
    plt.title(aasta + ' : ' + pealkiri)
    plt.show()

In [ ]:
# Lõpp - COL 9 - Mil viisil enamasti osalesite loengutes semestri alguses (1.-6. nädal)?
# Lõpp - COL10 - Mil viisil enamasti osalesite loengutes semestri keskel (7.-12. nädal)?
# Lõpp - COL11 - Mil viisil enamasti osalesite loengutes semestri lõpus (13.-16. nädal)?

tabel_abs_prot(9, 3, ['1.-6. nädal', '7.-12. nädal', '13.-16. nädal'], 'Mil viisil enamasti osalesite loengutes?')

In [ ]:
# Lõpp - COL 12 - Miks te just nii käitusite? Kui semestri jooksul viisi muutsite, siis miks?
# koosta_sõnapilv(df_lõpp, 12, True, 'loeng loengu video koha kohal loengut loengus loengute loengutes sest ning siis video videot kuna')
koosta_sõnapilv(df_lõpp, 12, True, '', True)

In [ ]:
sort_skaala = ['ei käinud loengus', 'pole üldse nõus', '1', '2', '3', '4', '5', '6', '7', 'olen täiesti nõus']

def col_stat_lõpp(tulp):
    data = {}
    tulba_nimi = df_lõpp[aastad[0]].columns[tulp]
    for aasta in aastad:
        df = df_lõpp[aasta]
        df = df.replace(['0', 0], 'ei käinud loengus') # 2019 aastal on selle koha peal 0
        df = df.fillna("Andmed puuduvad") # np.nan asendame, et andmed puuduvad
        data[aasta] = df[tulba_nimi].value_counts() # saab ka df.groupby(tulp)[tulp].agg('count')
    
        if 'Andmed puuduvad' in data[aasta]: # 2018 on numbrilised andmed Moodle'st kaotsi läinud, siis puuduvaid andmeid ei vaata
            data[aasta] = data[aasta].drop('Andmed puuduvad')
    
        data[aasta + ' %'] = round(data[aasta] / df_lõpp[aasta].shape[0]  * 100, 1) # vana kõikide vastustega: round(data[aasta] / data[aasta].sum() * 100, 1)

    df = pd.DataFrame(data)
    df = df.reindex(sorted(df.index, key=lambda x: sort_skaala.index(x))) # siin teeme uue sorditud indexi vaid leiduvatest ridadest
    return df, tulba_nimi

In [ ]:
# Lõpp - COL 13 - Loeng on tänapäeval programmeerimise kursusel mõttetu.

# NB!!!! Siit edasi on osa 2018 aasta andmeid puudu (väidetavalt moodlest kaotsi läinud)
# Reimo poolt antud skaala: (0) ei käinud loengus, (1) pole üldse nõus, ..., (7) olen täiesti nõus

df, tiitel = col_stat_lõpp(13)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# Lõpp - COL 14 - Loeng oleks kasulikum kui seda korraldataks teistmoodi.

df, tiitel = col_stat_lõpp(14)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# Lõpp - COL 15 - Soovi korral kirjutage kommentaare loengu kohta.
koosta_sõnapilv(df_lõpp, 15, True, '', True)

In [ ]:
# Lõpp - COL 16 - Käisin loengus või vaatasin loengu videot põhiliselt selleks, et punkte saada.

df, tiitel = col_stat_lõpp(16)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# Lõpp - COL 17 - Käiksin loengus või vaataksin loengu videot ka siis, kui selle eest üldse punkte ei antaks.

df, tiitel = col_stat_lõpp(17)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest', (8, 4), True, '', None)

In [ ]:
# Lõpp - COL 18 - Käiksin loengus või vaataksin loengu videot tihedamalt, kui punkte antaks rohkem.

df, tiitel = col_stat_lõpp(18)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# Lõpp - COL 19 - Tegin loengus (või loengu videot vaadates) märkmeid.

df, tiitel = col_stat_lõpp(19)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# Lõpp - COL 20 - Soovi korral täpsustage, kuidas tegite märkmeid loengus või loengu videot vaadates.
koosta_sõnapilv(df_lõpp, 20, True, '', True)

In [ ]:
# Lõpp - COL 21 - Oleksin rohkem märkmeid teinud, kui nutiseadmed (telefonid, arvutid) oleksid olnud lubatud

df, tiitel = col_stat_lõpp(21)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# Lõpp - COL 22 - Oleksin  nutiseadmeid (telefone, arvuteid) kasutanud asjakohaselt, kui kasutamine oleks lubatud.

df, tiitel = col_stat_lõpp(22)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# Lõpp - COL 23 - Oleksin nutiseadmeid (telefone, arvuteid) kasutanud kõrvalisteks tegevusteks, kui kasutamine oleks lubatud.

df, tiitel = col_stat_lõpp(23)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest', (8, 4), True, '', None)

In [ ]:
# Lõpp - COL 24 - Paljud üliõpilased loengus siiski kasutasid nutiseadmeid (telefone, arvuteid).

df, tiitel = col_stat_lõpp(24)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# Lõpp - COL 25 - Soovi korral kommenteerige nutiseadmete (telefonide, arvutite) lubamist/keelamist.
koosta_sõnapilv(df_lõpp, 25, True, '', True)

In [ ]:
# Lõpp - COL 26 - Klikkerite kasutamine oli asjakohane.

df, tiitel = col_stat_lõpp(26)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# Lõpp - COL 27 - Klikkerite kasutamine aitas vältida tähelepanu hajumist.

df, tiitel = col_stat_lõpp(27)
tiitel = 'Klikkerite kasutamine aitas vältida tähelepanu hajumist.'
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# Lõpp - COL 28 - Klikkerite kasutamine aitas tõsta huvi loengus võetud teema vastu.

df, tiitel = col_stat_lõpp(28)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# Lõpp - COL 29 - Klikkerid aitasid kaasa kriitilise mõtlemise arendamisel.

df, tiitel = col_stat_lõpp(29)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# Lõpp - COL 30 - Klikkerite kasutus julgustas loengutes osalema.

df, tiitel = col_stat_lõpp(30)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# Lõpp - COL 31 - Loengu videosse võiks olla integreeritud küsimusi, millele saab interaktiivselt vastata.

df, tiitel = col_stat_lõpp(31)
plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')

In [ ]:
# COL 32 - Soovi korral kommenteerige klikkerite kasutamist
# koosta_sõnapilv(df_lõpp, 32, True, 'klikker klikkeri klikkerit klikkerid')
koosta_sõnapilv(df_lõpp, 32, True, '', True)

In [ ]:
# COL 33 - Palun lisage veel kommentaare, mis aitaksid kursust arendada.
koosta_sõnapilv(df_lõpp, 33, True, '', True)

In [ ]:
# # TESTIME grupeeritud vastuseid

# def loe_andmed_grupp(leht, lisa_hinded = False):
#     df = {}
#     for aasta in aastad:
#         xlsx = pd.ExcelFile(aasta + ' grupp.xlsx')
#         df_l = pd.read_excel(xlsx, leht)
#         df_l = df_l.fillna('Andmed puuduvad')
#         if lisa_hinded:
#             df_h = pd.read_excel(xlsx, 'Hinded')
#             df_h = df_h.replace(to_replace = '-', value = 0)
#             df_h['Kasutaja täisnimi'] = df_h.Eesnimi + ' ' + df_h.Perenimi
#             df[aasta] = pd.merge(df_l, df_h, on = 'Kasutaja täisnimi', how='left').fillna(0) # ja paneme need kokku
#         else:
#             df[aasta] = df_l
# #         len(df[aasta][df[aasta].duplicated(['Kasutaja täisnimi'])]) # duplikaatide kontroll
#     return df

# df_grupp = loe_andmed_grupp('Lõpp')

# def col_stat_grupp(tulp, tükelda = False, sordi_index = False):
#     data = {}
#     tulba_nimi = df_grupp[aastad[0]].columns[tulp]
#     for aasta in aastad:
#         df = df_grupp[aasta]
#         df = df.replace(['0', 0], np.nan) # 0 andmed asendame np.nan
#         df = df.fillna("Andmed puuduvad") # np.nan asendame, et andmed puuduvad
#         if tükelda: # kui ühes lahtris on mitmed andmed, siis tükeldame
#             tükid = []
#             for item in df[tulba_nimi]:
#                 tükid += item.split(',')
#                 tükid_uus = [item.strip().lower() for item in tükid]
#                 data[aasta] = pd.Series(Counter(tükid_uus))
#         else:
#             data[aasta] = df[tulba_nimi].value_counts() # saab ka df.groupby(tulp)[tulp].agg('count')
            
#         data[aasta + ' %'] = round(data[aasta] / data[aasta].sum() * 100, 1)
# #         data[aasta] = map(int, list(data[aasta]))

#     df = pd.DataFrame(data)
    
#     if sordi_index:
#         df.sort_index(inplace = True) # sordime indexi järgi
#     else:
#         df.sort_values(by=[aastad[-1] + ' %'], inplace=True, ascending=False) # sordime viimase aasta järgi
#     return df, tulba_nimi

In [ ]:
# #df_grupp['2018']['KOKKUVÕTE']
# df, tiitel = col_stat_grupp(4, True)
# plot_barh_perc(df, tiitel, 'vastuste % kõigist vastusest')
# display(df.index)